## ETL Project

By: Chong Vang, Scott Weber, Ryan Porritt 

Create a blended rating from two different movie sources (Rotten Tomatoes and OMDB) and store results in a Postgre database

In [6]:
import requests
import json
import pandas as pd
from sqlalchemy import create_engine

In [7]:
from pprint import pprint

## CSV Extraction

In [8]:
csv_file = "Resources/Movies(2007-2011).csv"
movie_data_df = pd.read_csv(csv_file)
movie_data_df.head()

,Film,Genre,Lead Studio,Audience score %,Profitability,Rotten Tomatoes %,Worldwide Gross,Year
0,Zack and Miri Make a Porno,Romance,The Weinstein Company,70,1.747542,64,$41.94,2008
1,Youth in Revolt,Comedy,The Weinstein Company,52,1.090000,68,$19.62,2010
2,You Will Meet a Tall Dark Stranger,Comedy,Independent,35,1.211818,43,$26.66,2010
3,When in Rome,Comedy,Disney,44,0.000000,15,$43.04,2010
4,What Happens in Vegas,Comedy,Fox,72,6.267647,28,$219.37,2008


In [10]:
new_movie_data_df = movie_data_df[['Film', 'Year', 'Rotten Tomatoes %']].copy()
new_movie_data_df.head()

,Film,Year,Rotten Tomatoes %
0,Zack and Miri Make a Porno,2008,64
1,Youth in Revolt,2010,68
2,You Will Meet a Tall Dark Stranger,2010,43
3,When in Rome,2010,15
4,What Happens in Vegas,2008,28


## Rotten Tomatoes DataFrame

In [11]:
new_movie_data_df = new_movie_data_df.rename(columns={"Rotten Tomatoes %":"rotten_tomatoes_ratings", "Film":"film", "Year":"year"})
new_movie_data_df

,film,year,rotten_tomatoes_ratings
0,Zack and Miri Make a Porno,2008,64
1,Youth in Revolt,2010,68
2,You Will Meet a Tall Dark Stranger,2010,43
3,When in Rome,2010,15
4,What Happens in Vegas,2008,28
...,...,...,...
72,Across the Universe,2007,54
73,A Serious Man,2009,89
74,A Dangerous Method,2011,79
75,27 Dresses,2008,40


In [13]:
n = new_movie_data_df["rotten_tomatoes_ratings"]
type(n[0])

numpy.int64

In [14]:
new_movie_data_df.count()

film                       77
year                       77
rotten_tomatoes_ratings    77
dtype: int64

## Create a list of movie titles

In [16]:
list_of_files = list(new_movie_data_df["film"])
list_of_files

['Zack and Miri Make a Porno',
 'Youth in Revolt',
 'You Will Meet a Tall Dark Stranger',
 'When in Rome',
 'What Happens in Vegas',
 'Water For Elephants',
 'WALL-E',
 'Waitress',
 'Waiting For Forever',
 "Valentine's Day",
 "Tyler Perry's Why Did I get Married",
 'Twilight: Breaking Dawn',
 'Twilight',
 'The Ugly Truth',
 'The Twilight Saga: New Moon',
 "The Time Traveler's Wife",
 'The Proposal',
 'The Invention of Lying',
 'The Heartbreak Kid',
 'The Duchess',
 'The Curious Case of Benjamin Button',
 'The Back-up Plan',
 'Tangled',
 'Something Borrowed',
 "She's Out of My League",
 'Sex and the City Two',
 'Sex and the City 2',
 'Sex and the City',
 'Remember Me',
 'Rachel Getting Married',
 'Penelope',
 'P.S. I Love You',
 'Over Her Dead Body',
 'Our Family Wedding',
 'One Day',
 'Not Easily Broken',
 'No Reservations',
 "Nick and Norah's Infinite Playlist",
 "New Year's Eve",
 'My Week with Marilyn',
 'Music and Lyrics',
 'Monte Carlo',
 'Miss Pettigrew Lives for a Day',
 'Midnig

## Create an API call for OMDb

In [17]:
url = "http://www.omdbapi.com/?t="
api_key = "&apikey=trilogy"

## Commented code is part of the process

In [ ]:
#response = requests.get(url + "Zack and Miri Make a Porno" + api_key)
#print(response.url)

In [ ]:
#movie = response.json()
#pprint(movie)

In [ ]:
#title = movie["Title"]
#title

In [ ]:
# Isolate "data items" for easy reading
#ratings = movie["Ratings"]
#imdb_ratings = ratings[0]
#imdb_ratings = imdb_ratings["Value"]
#imdb_ratings

In [ ]:
#year = movie["Year"]
#year

In [18]:
movie_json = []
movie_title = []
movie_ratings = []
movie_year = []

## Extract JSON files from API using OMDB movie list

In [19]:
for film in list_of_files:
    movie = requests.get(url + film + api_key).json()
    
    movie_json.append(movie)
    
    #add the rating to the list
    #ratings = movie["Ratings"]
    #imdb_ratings = ratings[0]
    #imdb_ratings = imdb_ratings["Value"]
    #movie_ratings.append(imdb_ratings)
    
    #add the year to the list
    #year = movie["Year"]
    #movie_year.append(year)
    
    

In [ ]:
#len(movie_json)

In [ ]:
#movie_json[0]["Ratings"][0]["Value"]

In [ ]:
#movie_json[11]["Ratings"][0]["Value"]

## Filter title, year, ratings

In [20]:
for i in range(77):
    try:
        movie_ratings.append(movie_json[i]["Ratings"][0]["Value"])
        
    except:
        print("ratings")
        print(f"The error: {list_of_files[i]}")
        print(i)
        movie_ratings.append("NaN")
    
    try:
        movie_title.append(movie_json[i]["Title"])
    
    except:
        print("title")
        print(f"The error: {list_of_files[i]}")
        print(i)
        movie_title.append("NaN")
        
    try:
        movie_year.append(movie_json[i]["Year"])
        
    except:
        print("year")
        print(f"The error: {list_of_files[i]}")
        print(i)
        movie_year.append("NaN")
    

ratings
The error: Tyler Perry's Why Did I get Married
10
title
The error: Tyler Perry's Why Did I get Married
10
year
The error: Tyler Perry's Why Did I get Married
10
ratings
The error: Sex and the City Two
25
title
The error: Sex and the City Two
25
year
The error: Sex and the City Two
25
ratings
The error: Marley and Me
44
title
The error: Marley and Me
44
year
The error: Marley and Me
44
ratings
The error: High School Musical 3: Senior Year
60
title
The error: High School Musical 3: Senior Year
60
year
The error: High School Musical 3: Senior Year
60
ratings
The error: Gnomeo and Juliet
64
title
The error: Gnomeo and Juliet
64
year
The error: Gnomeo and Juliet
64
ratings
The error: Gnomeo and Juliet
65
title
The error: Gnomeo and Juliet
65
year
The error: Gnomeo and Juliet
65


In [ ]:
#movie_ratings

## IMDB DataFrame

In [23]:
film_data_df = pd.DataFrame(list(zip(movie_title, movie_year, movie_ratings)), columns = ['film', 'year', 'imdb_ratings'])
film_data_df.tail(50)

,film,year,imdb_ratings
27,Sex and the City,2008,5.6/10
28,Remember Me,2010,7.1/10
29,Rachel Getting Married,2008,6.7/10
30,Penelope,2006,6.7/10
31,P.S. I Love You,2007,7.0/10
32,Over Her Dead Body,2008,5.2/10
33,Our Family Wedding,2010,5.0/10
34,One Day,2011,7.0/10
35,Not Easily Broken,2009,6.1/10
36,No Reservations,2007,6.3/10


In [26]:
film_data_df["imdb_ratings"] = film_data_df["imdb_ratings"].str.split("/", n =1, expand = True)
film_data_df["imdb_ratings"]

0     6.5
1     6.4
2     6.3
3     5.6
4     6.1
     ... 
72    7.3
73    7.0
74    6.4
75    6.1
76    7.8
Name: imdb_ratings, Length: 77, dtype: object

In [ ]:
film_data_df

## Transform Data

In [27]:
film_data_df["imdb_ratings"] = film_data_df["imdb_ratings"].astype(float)
film_data_df["imdb_ratings"] = film_data_df["imdb_ratings"] * 10
film_data_df.head(12)

,film,year,imdb_ratings
0,Zack and Miri Make a Porno,2008,65.0
1,Youth in Revolt,2009,64.0
2,You Will Meet a Tall Dark Stranger,2010,63.0
3,When in Rome,2010,56.0
4,What Happens in Vegas,2008,61.0
5,Water for Elephants,2011,69.0
6,WALL·E,2008,84.0
7,Waitress,2007,70.0
8,Waiting for Forever,2010,60.0
9,Valentine's Day,2010,57.0


In [28]:
film_data_df= film_data_df.dropna()

## Make IMDB ratings field an integer

In [29]:
film_data_df["imdb_ratings"] = film_data_df["imdb_ratings"].astype(int)
film_data_df.head(12)

#df.round(0).astype(int)

C:\Users\Ryanj\Anaconda3\envs\PythonWebMongo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,film,year,imdb_ratings
0,Zack and Miri Make a Porno,2008,65
1,Youth in Revolt,2009,64
2,You Will Meet a Tall Dark Stranger,2010,63
3,When in Rome,2010,56
4,What Happens in Vegas,2008,61
5,Water for Elephants,2011,69
6,WALL·E,2008,84
7,Waitress,2007,70
8,Waiting for Forever,2010,60
9,Valentine's Day,2010,57


In [30]:
n = list(film_data_df["imdb_ratings"])
type(n[0])


int

### Connect to local database

In [35]:
rds_connection_string = "ryanporrit:Helios007()@localhost:5432/ETL_Film_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [36]:
engine.table_names()

['rottentomatoes', 'imdb']

## Load

### Use pandas to load csv converted DataFrame into database

In [37]:
new_movie_data_df.to_sql(name='rottentomatoes', con=engine, if_exists='append', index=False)

### Use pandas to load json converted DataFrame into database

In [38]:
film_data_df.to_sql(name='imdb', con=engine, if_exists='append', index=False)

In [ ]:
### Confirm data has been added by querying the customer_name table
#* NOTE: can also check using pgAdmin